In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px


In [ ]:
def load_sheet(file_id, gid):
    url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gid}"
    df = pd.read_csv(url)
    return df

# Charger chaque onglet dans un DataFrame
file_id_avis_de_mission = "1iEPI3UaPmMzDF1SpHAmR6ghlScBH1gC3FFWdn8MLH8U"
file_id_impact_cp="10eH5dsJ2-t_ppLPgr9TIrijOR_1DdOYEPhSSiP6BhrI"
gid = {
    "avis_de_mission": "90394237",
    "impact_cp": "0"
}

https://docs.google.com/spreadsheets/d/1iEPI3UaPmMzDF1SpHAmR6ghlScBH1gC3FFWdn8MLH8U/edit?gid=90394237#gid=90394237

In [146]:
def nettoyer_data_avis_de_mission(file_id, gid, verbose=False, colonnes_a_supprimer=None, colonnes_essentielles=None, nettoyer_texte=True):
    if colonnes_a_supprimer is None:
        colonnes_a_supprimer = ['N° D\'intervention', 'N°']
    if colonnes_essentielles is None:
        colonnes_essentielles = ['Nom / Prénom', 'Date d\'intervention']

    # Charger la feuille
    try:
        url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gid}"
        data_avis_de_mission = pd.read_csv(url)
    except Exception as e:
        print(f"Erreur lors du chargement des données : {e}")
        return None

    if verbose:
        print(f"Données chargées : {data_avis_de_mission.shape}")

    # Conversion des dates
    data_avis_de_mission["Date d'intervention"] = pd.to_datetime(
        data_avis_de_mission["Date d'intervention"], format='%d/%m/%Y', errors='coerce'
    ).dt.date

    # Supprimer les colonnes inutiles
    data_avis_de_mission = data_avis_de_mission.drop(columns=colonnes_a_supprimer, errors='ignore')

    # Supprimer les lignes où des colonnes essentielles ont des valeurs manquantes
    data_avis_de_mission = data_avis_de_mission.dropna(subset=colonnes_essentielles)

    # Conversion des colonnes numériques
    for col in ['Nombre de jour', 'Année scolaire 2024/2025']:
        if col in data_avis_de_mission.columns:
            data_avis_de_mission[col] = data_avis_de_mission[col].fillna(0).astype(int)

    # Renommer les colonnes
    nouveaux_noms = {
        'Nom / Prénom': 'Nom',
        'Discipline': 'Discipline',
        'Lieu d\'intervention': 'Lieu',
        'Date d\'intervention': 'Date',
        'Nombre de jour': 'nb_jours',
        'Année scolaire 2024/2025': 'AnneeScolaire',
        'Mission': 'Mission',
        'Objet de la mission': 'Objet',
        'Situation': 'Situation'
    }
    data_avis_de_mission = data_avis_de_mission.rename(columns=nouveaux_noms)

    # Nettoyage des colonnes textuelles
    if nettoyer_texte:
        for col in data_avis_de_mission.select_dtypes(include=['object']).columns:
            data_avis_de_mission[col] = (
                data_avis_de_mission[col]
                .astype(str)
                .str.strip()
                .replace('', pd.NA)
                .str.lower()
            )

    if verbose:
        print(f"Données nettoyées : {data_avis_de_mission.shape}")

    return data_avis_de_mission

df_avis_de_mission=nettoyer_data_avis_de_mission(file_id_avis_de_mission, gid['avis_de_mission'],verbose=False, colonnes_a_supprimer=None, colonnes_essentielles=None, nettoyer_texte=True)

In [147]:
# Nombre total de missions réalisées
def calcul_total_missions(df):
    """
    Calcule le nombre total de missions réalisées.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.

    Returns:
        int: Nombre total de missions réalisées.
    """
    return df.shape[0]

# Durée moyenne des missions
def duree_moyenne_missions(df):
    """
    Calcule la durée moyenne des missions réalisées.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.

    Returns:
        float: Durée moyenne des missions en jours.
    """
    return df['nb_jours'].mean()

# Appel des fonctions
total_missions = calcul_total_missions(df_avis_de_mission)
duree_moyenne = duree_moyenne_missions(df_avis_de_mission)

# Affichage des résultats
print(f"Nombre total de missions réalisées : {total_missions}")
print(f"Durée moyenne des missions : {duree_moyenne:.2f} jours")


Nombre total de missions réalisées : 112
Durée moyenne des missions : 1.33 jours


In [148]:
import plotly.express as px

def visualiser_missions_par_cp(df):
    """
    Calcule et affiche un bar chart du nombre total de missions réalisées par CP.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    # Calcul du nombre de missions par CP
    df_missions_cp = df['Nom'].value_counts().reset_index()
    df_missions_cp.columns = ['CP', 'Nombre de missions']

    # Création du graphique avec Plotly
    fig = px.bar(
        df_missions_cp,
        x='CP',
        y='Nombre de missions',
        title="Nombre total de missions réalisées par CP",
        text='Nombre de missions',
        color_discrete_sequence=px.colors.qualitative.T10)

    # Mise en forme du graphique
    fig.update_traces(textposition='outside')
    fig.update_layout(showlegend=False)
    # Affichage du graphique
    fig.update_xaxes(title="")
    fig.show()

visualiser_missions_par_cp(df_avis_de_mission)


In [149]:
def visualiser_repartition_par_degre(df):
    """
    Affiche un pie chart de la répartition des missions par degré (1/2).

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    # Calcul de la répartition par degré
    repartition_degre = df['Mission'].value_counts().reset_index()
    repartition_degre.columns = ['Degré', 'Nombre de missions']

    # Mapping des valeurs pour afficher Degré 1 / Degré 2
    repartition_degre['Degré'] = repartition_degre['Degré'].map({'cp1d': 'Degré 1', 'cp2d': 'Degré 2'})

    # Création du graphique avec Plotly
    fig = px.pie(
        repartition_degre,
        names='Degré',
        values='Nombre de missions',
        title="Répartition des missions par degré",
        color_discrete_sequence=px.colors.qualitative.T10
    )

    # Affichage du graphique
    fig.show()

    # Appel de la fonction
visualiser_repartition_par_degre(df_avis_de_mission)


In [150]:

def nb_missions_par_etablissement_par_eleve(df):
    """
    Affiche un bar chart du ratio missions/élèves pour chaque établissement.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions,
                           incluant une colonne 'Nombre d'élèves'.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    # Calcul du nombre total de missions par établissement
    missions_par_etablissement = df['Lieu'].value_counts().reset_index()
    missions_par_etablissement.columns = ['Établissement', 'Nombre de missions']

    # Ajout du nombre d'élèves à partir des données préexistantes
    nombre_eleves = {
        'agadir': 1400, 'lfiad': 884, 'lfilm': 4319, 'tanger': 882,
        'dakhla': 263, 'lfijc': 601, 'essaouira': 60, 'laayoune': 260,
        'gs majorelle': 603, 'lfiam': 2014, 'oujda': 336
    }
    missions_par_etablissement['Nombre d\'élèves'] = missions_par_etablissement['Établissement'].map(nombre_eleves)

    # Calcul du ratio missions/élèves
    missions_par_etablissement['Ratio missions/élèves'] = (
        missions_par_etablissement['Nombre de missions'] / missions_par_etablissement['Nombre d\'élèves']
    )
    # Trier par ordre décroissant du ratio missions/élèves
    missions_par_etablissement = missions_par_etablissement.sort_values(by='Ratio missions/élèves', ascending=False)


    # Création du graphique avec Plotly
    fig = px.bar(
        missions_par_etablissement,
        x='Établissement',
        y='Ratio missions/élèves',
        title="Ratio missions par établissement et par élève",
        text='Ratio missions/élèves',
        color_discrete_sequence=px.colors.qualitative.G10  # Palette de couleurs
    )

    # Mise en forme du graphique
    fig.update_traces(textposition='outside', texttemplate='%{text:.2f}')
    fig.update_layout(
        title=dict(x=0.5),  # Centrer le titre
        xaxis=dict(title=""),
        yaxis=dict(title="Ratio missions/élèves")
    )

    # Affichage du graphique
    fig.show()

nb_missions_par_etablissement_par_eleve(df_avis_de_mission)


In [151]:

def nb_missions_par_etablissement(df):
    """
    Affiche un bar chart du nombre total de missions réalisées par établissement.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    # Calcul du nombre de missions par établissement
    missions_par_etablissement = df['Lieu'].value_counts().reset_index()
    missions_par_etablissement.columns = ['Établissement', 'Nombre de missions']

    # Trier les établissements par nombre de missions (ordre décroissant)
    missions_par_etablissement = missions_par_etablissement.sort_values(by='Nombre de missions', ascending=False)

    # Création du graphique avec Plotly
    fig = px.bar(
        missions_par_etablissement,
        x='Établissement',
        y='Nombre de missions',
        title="Nombre de missions par établissement",
        text='Nombre de missions',
        color_discrete_sequence=[px.colors.qualitative.G10[1]] # Palette de couleurs
    )

    # Mise en forme du graphique
    fig.update_traces(textposition='outside', texttemplate='%{text}')
    fig.update_layout(
        title=dict(x=0.5),  # Centrer le titre
        xaxis=dict(title=""),
        yaxis=dict(title="Nombre de missions")
    )

    # Affichage du graphique
    fig.show()

nb_missions_par_etablissement(df_avis_de_mission)


# VUE PAR CP

In [236]:
def total_missions_cp(df, nom_cp):
    """
    Calcule le nombre total de missions réalisées pour un CP donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        nom_cp (str): Le nom du CP.

    Returns:
        int: Nombre total de missions réalisées pour le CP.
    """
    return df[df['Nom'] == nom_cp].shape[0]

total_missions_cp(df_avis_de_mission, "clélia parra")

6

In [241]:
def duree_moyenne_cp(df, nom_cp):
    """
    Calcule la durée moyenne des missions réalisées pour un CP donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        nom_cp (str): Le nom du CP.

    Returns:
        float: Durée moyenne des missions pour le CP.
    """
    df_cp = df[df['Nom'] == nom_cp]
    moyenne = df_cp['nb_jours'].mean()
    return float(round(moyenne, 1))

duree_moyenne_cp(df_avis_de_mission, "clélia parra")

1.3

In [235]:
def repartition_par_etablissement_cp(df, nom_cp):
    """
    Affiche un bar chart de la répartition des missions par établissement pour un CP donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        nom_cp (str): Le nom du CP.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    df_cp = df[df['Nom'] == nom_cp]
    missions_par_etablissement = df_cp['Lieu'].value_counts().reset_index()
    missions_par_etablissement.columns = ['Établissement', 'Nombre de missions']

    fig = px.bar(
        missions_par_etablissement,
        x='Nombre de missions',
        y='Établissement',
        title=f"Répartition des missions par établissement pour {nom_cp}",
        text='Nombre de missions',
        orientation='h',
        color_discrete_sequence=px.colors.qualitative.G10
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(
        title=dict(x=0.5),
        xaxis_title="Nombre de missions",
        yaxis_title="")
    fig.show()

repartition_par_etablissement_cp(df_avis_de_mission, "clélia parra")


In [250]:
def repartition_par_objet_cp(df, nom_cp):
    """
    Affiche un pie chart de la répartition des missions par objet pour un CP donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        nom_cp (str): Le nom du CP.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    df_cp = df[df['Nom'] == nom_cp]
    missions_par_objet = df_cp['Objet'].value_counts().reset_index()
    missions_par_objet.columns = ['Objet', 'Nombre de missions']

    fig = px.pie(
        missions_par_objet,
        names='Objet',
        values='Nombre de missions',
        title=f"Répartition des missions par objet pour {nom_cp}",
        hole=0.4,
        color_discrete_sequence=px.colors.qualitative.G10
    )
    fig.update_layout(
        legend=dict(
            orientation="h",  # Orientation horizontale
            yanchor="top",
            y=-0.1,  # Position sous le graphique
            xanchor="center",
            x=0.5  # Centrer la légende
        )
    )
    fig.show()
repartition_par_objet_cp(df_avis_de_mission, "clélia parra")

# VUE PAR ETABLISSEEMNT

In [251]:
def total_missions_etablissement(df, etablissement):
    """
    Calcule le nombre total de missions réalisées dans un établissement donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        etablissement (str): Le nom de l'établissement.

    Returns:
        int: Nombre total de missions réalisées dans l'établissement.
    """
    return df[df['Lieu'] == etablissement].shape[0]


total_missions_etablissement(df_avis_de_mission,'lfiam')

15

In [274]:
def repartition_par_objet_et_degre(df, etablissement):
    """
    Affiche un stacked bar chart des missions par objets et par degré pour un établissement donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        etablissement (str): Le nom de l'établissement.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    df_etab = df[df['Lieu'] == etablissement].copy()  # Création d'une copie explicite
    df_etab['Degré'] = df_etab['Mission'].apply(lambda x: "Degré 1" if "cp1" in x.lower() else "Degré 2")

    repartition = df_etab.groupby(['Objet', 'Degré'])['Nom'].count().reset_index()
    repartition.columns = ['Objet', 'Degré', 'Nombre de missions']

    fig = px.bar(
        repartition,
        x='Objet',
        y='Nombre de missions',
        color='Degré',
        title=f"Répartition des missions par objets et par degré pour {etablissement}",
        barmode='stack',
        text='Nombre de missions'
    )
    fig.update_traces(textposition='outside')
    fig.update_layout(
        title=dict(x=0.5),
        xaxis_title="",
        yaxis_title="Nombre de missions",
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1, title=None),
        xaxis=dict(tickangle=25)
    )
    fig.show()


repartition_par_objet_et_degre(df_avis_de_mission,'lfiam')


In [257]:
def repartition_par_degre_etablissement(df, etablissement):
    """
    Affiche un pie chart de la répartition des missions par degré pour un établissement donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        etablissement (str): Le nom de l'établissement.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    df_etab = df[df['Lieu'] == etablissement].copy()  # Création d'une copie explicite
    df_etab['Degré'] = df_etab['Mission'].apply(lambda x: "Degré 1" if "cp1" in x.lower() else "Degré 2")

    repartition = df_etab['Degré'].value_counts().reset_index()
    repartition.columns = ['Degré', 'Nombre de missions']

    fig = px.pie(
        repartition,
        names='Degré',
        values='Nombre de missions',
        title=f"Répartition des missions par degré pour {etablissement}",
        hole=0.4
    )

    fig.update_layout(
        title=dict(x=0.5),
        legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5)
    )
    fig.show()

repartition_par_degre_etablissement(df_avis_de_mission,'lfiam')


In [260]:
def repartition_degre2_par_discipline(df, etablissement):
    """
    Affiche un pie chart de la répartition des missions de degré 2 par discipline pour un établissement donné.

    Args:
        df (pd.DataFrame): Le DataFrame contenant les données des missions.
        etablissement (str): Le nom de l'établissement.

    Returns:
        None: Affiche directement le graphique avec Plotly.
    """
    df_etab = df[df['Lieu'] == etablissement].copy()  # Création d'une copie explicite
    df_etab_degre2 = df_etab[df_etab['Mission'].str.contains("cp2", case=False)].copy()  # Création d'une copie explicite

    repartition = df_etab_degre2['Discipline'].value_counts().reset_index()
    repartition.columns = ['Discipline', 'Nombre de missions']

    fig = px.pie(
        repartition,
        names='Discipline',
        values='Nombre de missions',
        title=f"Répartition des missions de degré 2 par discipline pour {etablissement}",
        hole=0.4
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(
        title=dict(x=0.5),
        legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5)
    )
    fig.show()

repartition_degre2_par_discipline(df_avis_de_mission,'lfiam')
